## Abstract
- In this notebook, I will introduce very simple example of 'LightGBMTunerCV' from 'optuna.integration.lightgbm'.
- Because 'it tunes the important hyperparameter variables in order', we can reduce the number of trials for tuning.
- For more details, see [official documant](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.lightgbm.LightGBMTunerCV.html) and [this blog](https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258)
- Finnaly I got CV = 0.6974 !

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb

In [ ]:
# Load data
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
print('Train size:', train.shape)
print('Test size:', test.shape)

print('Train data info: ')
display(train.head())
train.info()

### Standarization
- Before tuning, let's standardize explain variables.
- In this competition, all of the columns in given data are continuous.

In [ ]:
features = [f'cont{i}' for i in range(1, 15)]

scaler = StandardScaler()
scaler.fit(train[features])

# transform step
train[features] = scaler.transform(train[features])
test[features] = scaler.transform(test[features])

In [ ]:
# separate features and target
X_train = train[features]
y_train = train['target']

X_test = test[features]

## Tuning
'LightGBMTunerCV' is a hyperparameter tuner for LightGBM with cross-validation.

In [ ]:
%%time

dtrain = lgb.Dataset(X_train, label=y_train)

params = {
    "objective": "regression", 
    "metric": "rmse", 
    "boosting_type": "gbdt", 
    "seed": 1
        }

tuner = lgb.LightGBMTunerCV(
    params, dtrain, verbose_eval=False, early_stopping_rounds=10, 
    num_boost_round=10000, folds=KFold(n_splits=5, shuffle=True, random_state=1), seed=1
)
tuner.run()

## Tuning Result

In [ ]:
print("Best score:", tuner.best_score)
best_params = tuner.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

## Discussion
- As described above, I used features as it was. Better score is expected by adequate feature engineering.
- In 'optuna.integration', several models are covered, for example, Keras, sklearn, XGBoost and so on. It's worth tryng them.